# 大纲

2022/05/22-

- [X] 树的基本知识点
    - [x] 表示和存储
    - [X] 常见的树的类型
- [X] 树常规操作的代码实现
- [X] 树的遍历算法代码实现
    - [X] 前序遍历
    - [X] 中序遍历
    - [X] 后序遍历
    - [X] 三者之间的关系
- [X] 深搜和广搜算法实现
- [ ] 迭代方法的专项练习
- [ ] 时空复杂度分析
- [ ] leetcode经典例题
- [ ] 我的总结
- [ ] 现实应用


参考链接：
- [leetcode cookbook tree](https://books.halfrost.com/leetcode/ChapterTwo/Tree/)
- [leetcode cookbook DFS](https://books.halfrost.com/leetcode/ChapterTwo/Depth_First_Search/)
- [leetcode cookbook BFS](https://books.halfrost.com/leetcode/ChapterTwo/Breadth_First_Search/)


# 树的基本知识

构成：父节点和子节点，有层级关系
存储和实现：链表的形式，节点存储自己的值，和子节点
常见的基本树的类型：
- 二叉树：一个父节点最多有两个子节点
- 二叉查找树BST：二叉树 + 左节点的所有值都小于父节点，右节点的所有值都大于父节点。操作
    - 插入/构建：从根节点开始，如果比当前节点小，就走向左节点；否则走向右节点。当要访问的节点不存在时，就创建。不看查找的过程，只看插入，复杂度O(1)；看全部过程，复杂度是O(logN)到O(N)的
    - 删除：删除操作要保证不破坏左右子节点和父节点的相对顺序。复杂度是O(logN)到O(N)的；不看查找的过程，只看删除，复杂度O(1)
        1. 找到删除该点后，下一个要放在这个位置的节点（不妨叫做候补节点）。
        2. 用候补节点的值替换该节点的值。
        3. 因为候补节点的左节点一定是空的，所以这一步一定可以替换，不用再循环。
        如何找到候补节点？
        - 如果该节点没有子节点，候补节点为None
        - 如果该节点只有一个子节点，则候补节点为该子节点
        - 如果该节点有两个子节点，则找到右子树的最左子节点。
    - 查找：复杂度是O(logN)到O(N)的（最坏情况，退化为链表；由此发展出了AVL，二叉平衡树）
    - 与堆的区别：
        - 定义上，（最小）堆是任意节点的值小于等于其子节点。二叉搜索树是：左节点小于父节点，父节点小于右节点。
        - 插入操作上：堆是先插入到末尾，再执行上浮操作：和更大的父节点交换。二叉搜索树是从父节点开始，判断进入左子树还是右子树，直到遇到空节点，再插入
        - 删除操作上：堆是把末尾元素插到头部，再执行下沉操作：和更小的子节点交换。二叉搜索树是找到要删除的点，判断左右子树是否为空。若都不为空，找到中序的后一个节点，进行交换。再对后一个节点的左右子节点进行移动处理。
- 更多类型：二叉平衡树、字典树、线段树、树状数组，见下一篇 树和图（二：树的进阶）

In [1]:
# Definition for a binary tree node.
class TreeNode(object):
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right
    def print(self):
        print(self.val)
        if self.left:
            print(str(self.val) + "->left: ")
            self.left.print()
        if self.right:
            print(str(self.val) + "->right: ")
            self.right.print()

In [22]:
# BST Binary Search Tree implementation

class BST(object):
    def __init__(self):
        self.root = TreeNode() # this is the node pointing to the root
        
    def insert(self, val):
        if self.root.left is None:
            self.root.left = TreeNode(val)
            self.root.val = val + 1
        parent_node = self.root.left
        while True:
            if parent_node.val > val:
                if parent_node.left is None:
                    parent_node.left = TreeNode(val)
                    break
                else:
                    parent_node = parent_node.left
            elif parent_node.val == val:
                break
            else:
                if parent_node.right is None:
                    parent_node.right = TreeNode(val)
                    break
                else:
                    parent_node = parent_node.right
                    
    def findNextBigger(self, node):
        val = node.val
        head = node
        child = head.right
        
        while True:
            if child.left:
                head = child
                child = child.left
            elif child.right:
                head = child
                child = child.right
            else:
                break
                
        return head, child  
        
    def delete(self, val):
        parent_node, _ = self.search(val)
        if parent_node.val > val:
            cur_node = parent_node.left
            if cur_node.left is None:
                parent_node.left = cur_node.right
            elif cur_node.right is None:
                parent_node.left = cur_node.left
            else:
                to_replace_parent_node, to_replace_node = self.findNextBigger(cur_node)
                cur_node.val = to_replace_node.val
                if to_replace_parent_node.right == to_replace_node:
                    to_replace_parent_node.right = to_replace_node.right
                else:
                    to_replace_parent_node.left = to_replace_node.right
                
        else:
            cur_node = parent_node.right
            if cur_node.left is None:
                parent_node.right = cur_node.right
            elif cur_node.right is None:
                parent_node.right = cur_node.left
            else:
                to_replace_parent_node, to_replace_node = self.findNextBigger(cur_node)
                cur_node.val = to_replace_node.val
                if to_replace_parent_node.right == to_replace_node:
                    to_replace_parent_node.right = to_replace_node.right
                else:
                    to_replace_parent_node.left = to_replace_node.right
        
    def search(self, val):
        # return two values, one is the parent node, the other is whether to find
        if self.root.left is None:
            return self.root, False
        parent_node = self.root
        cur_node = parent_node.left
        while True:
            if cur_node.val > val:
                if cur_node.left is None:
                    return cur_node, False
                else:
                    parent_node = cur_node
                    cur_node = cur_node.left
            elif cur_node.val == val:
                return parent_node, True
            else:
                if cur_node.right is None:
                    return cur_node, False
                else:
                    parent_node = cur_node
                    cur_node = cur_node.right
    def print(self):
        self.root.left.print()

In [23]:
tree = BST()
nums = [6,4,2,5,8,7,10]
for num in nums:
    tree.insert(num)
tree.print()
tree.delete(6)
print('After deleting node')
tree.print()

6
6->left: 
4
4->left: 
2
4->right: 
5
6->right: 
8
8->left: 
7
8->right: 
10
After deleting node
7
7->left: 
4
4->left: 
2
4->right: 
5
7->right: 
8
8->right: 
10


## 树的遍历

常见的遍历有四种，每种都有递归和迭代两种实现方法
- 递归：重复调用
- 迭代：遍历。常常用栈来辅助实现
分别是：
- 前序：中左右
    - [144Binary Tree Preorder Traversal](https://leetcode.cn/problems/binary-tree-preorder-traversal/)
    - 迭代：当前这一步做什么？将当前元素pop出来，存储值，再依次入栈右子节点，左子节点
- 中序：左中右
    - [94Binary Tree Inorder Traversal](https://leetcode.cn/problems/binary-tree-inorder-traversal/)
    - 🌟迭代：当前这一步做什么？先处理需要入栈的元素。把该元素入栈，把该元素的所有左子节点入栈。接着，出栈栈顶，存储值。把右子节点设置为下一个要入栈的元素。同前序，一个元素出栈的时候，就是存储值的时候，代表了其和其左子树都访问完了。不同前序：先把左子树入栈。两层循环
    - 迭代版本2:用一个标记来表明左子树是否已经入栈了，进而区分是执行存储值，还是入栈左子树。
- 后序：左右中
    - [145Binary Tree Postorder Traversal](https://leetcode.cn/problems/binary-tree-postorder-traversal/)
    - [590n-ary tree postorder traversal](https://leetcode.cn/problems/n-ary-tree-postorder-traversal/)
- 层序遍历：
    - [X] 和BFS有什么区别？层序遍历就是BFS了！
    - [102Binary tree level order traversal](https://leetcode.cn/problems/binary-tree-level-order-traversal/)
        - 思路一：每次遍历一个点。使用队列来维护待遍历的节点。对于每个节点，用一个状态记录其所在层数。当遍历进入下一层时，结果res加一个新的list。
        - 思路二：每次遍历一层。每层的结果是一个list

二叉树的前中后三者遍历之间的关系
从任意两种可以得到第三种；从任意两种可以恢复出一棵二叉树。恢复出二叉树的要点是：找到根节点，和左右子树的遍历结果，递归构造。

- 从前序、中序恢复出二叉树（唯一）：前序的第一个是根节点，在中序中找到根节点的位置，中序中根节点之前的都是左子树的遍历，根节点之后的都是右子树的遍历。通过中序中左子树的长度，去前序中截取左子树的前序遍历结果，递归调用构造左子树；右子树同理。
    - 参考例题：[(medium)105Construct Binary Tree From Preorder And Inorder Traversal](https://leetcode.cn/problems/construct-binary-tree-from-preorder-and-inorder-traversal/)
- 从前序、后序中恢复出二叉树（可能不唯一）：前序的第一个是根节点，在后序的最后一个；
    - 假设前序有左子树，则前序的第二个节点是左子树的根节点，去后序中找到该节点的位置，即可划分出左子树。进而在左右子树上递归调用
    - 假设前序没有左子树：那么前序的第二个节点是右子树的根节点，应该出现在后序的倒数第二个，如果不是出现在倒数第二个，则证明前序有左子树。因此对于只有单叶子节点的父节点，结果可能不唯一，无法区分该节点是左叶子节点，还是右叶子节点。
    - 参考例题：[(medium)889Construct Binary Tree From Preorder And Postorder Traversal](https://leetcode.cn/problems/construct-binary-tree-from-preorder-and-postorder-traversal/)
- 从中序、后序中恢复出二叉树（唯一）：后序的最后一个节点是叶子节点，在中序中找到根节点的位置，中序中根节点之前的都是左子树的遍历，根节点之后的都是右子树的遍历。递归调用。
    - 参考例题：[(medium)106Construct Binary Tree From Inorder And Postorder Traversal](https://leetcode.cn/problems/construct-binary-tree-from-inorder-and-postorder-traversal/)
    

In [ ]:
# preorder recursion version
class Solution:
    def preorderTraversal(self, root: Optional[TreeNode]) -> List[int]:
        if not root:
            return []
        res = [root.val] # 先访问父节点
        res.extend(self.preorderTraversal(root.left)) # 再访问左子节点
        res.extend(self.preorderTraversal(root.right)) # 最后访问右子节点
        return res

# preorder iteration version
class Solution:
    def preorderTraversal(self, root: Optional[TreeNode]) -> List[int]:
        if not root:
            return []
        stack = [root] # 模拟递归的栈，栈中存放的是下一个要访问的元素
        res = []
        while stack:
            top = stack.pop()
            res.append(top.val) # 先访问父节点
            if top.right:
                stack.append(top.right) # 先入栈右子节点，为了后出栈
            if top.left:
                stack.append(top.left) # 后入栈左子节点，先出栈
        return res

# n-ary preorder recursion version
class Solution:
    def preorder(self, root: 'Node') -> List[int]:
        if root is None:
            return []
        res = []
        res.append(root.val)
        for child in root.children:
            res.extend(self.preorder(child))
        return res

In [ ]:
# inorder recursion version
class Solution:
    def inorderTraversal(self, root: Optional[TreeNode]) -> List[int]:
        if not root:
            return []
        res = []
        res.extend(self.inorderTraversal(root.left))
        res.append(root.val)
        res.extend(self.inorderTraversal(root.right))
        return res 
    
# inorder iteration version
class Solution:
    def inorderTraversal(self, root: Optional[TreeNode]) -> List[int]:
        if not root:
            return []
        stack = []
        to_enter = root
        res = []
        while stack or to_enter:
            while to_enter:
                stack.append(to_enter)
                to_enter = to_enter.left
            head = stack.pop()
            res.append(head.val)
            to_enter = head.right
        return res 
    
# inorder iteration version 2   
class Solution:
    def inorderTraversal(self, root: Optional[TreeNode]) -> List[int]:
        if not root:
            return []
        stack = [(root, 0)] # 用一个标记来表明左子树是否已经入栈了
        res = []
        while stack:
            head = stack.pop()
            if head[1] == 0: # 入栈左子树
                to_enter = head[0].left
                stack.append((head[0], 1))
                while to_enter:
                    stack.append((to_enter, 0))
                    to_enter = to_enter.left
            else: # 存储值
                res.append(head[0].val)
                if head[0].right:
                    stack.append((head[0].right, 0))
        return res

In [ ]:
# post order recursion version
class Solution:
    def postorderTraversal(self, root: Optional[TreeNode]) -> List[int]:
        res = []
        if not root:
            return []
        res.extend(self.postorderTraversal(root.left))
        res.extend(self.postorderTraversal(root.right))
        res.append(root.val)
        return res

# postorder iteration version
class Solution:
    def postorderTraversal(self, root: Optional[TreeNode]) -> List[int]:
        res = []
        stack = []
        stack.append((root, 0)) # 标记法
        while stack:
            top, cmd = stack.pop()
            if top is None:
                continue
            if cmd == 0:
                stack.append((top, 1))
                stack.append((top.right, 0))
                stack.append((top.left, 0))
            else:
                res.append(top.val)
        return res

# n-ary postorder
class Solution:
    def postorder(self, root: 'Node') -> List[int]:
        if root is None:
            return []
        res = []
        for child in root.children:
            res.extend(self.postorder(child))
        res.append(root.val)
        return res

In [ ]:
# 102 level order traversal（BFS）思路一：每次遍历一个点
from collections import deque
class Solution:
    def levelOrder(self, root: TreeNode) -> List[List[int]]:
        if not root:
            return []
        cur_layer = -1
        q = deque([[root, 0]]) # 使用队列来维护待遍历的节点。记录一个状态：表示所在层数，
        res = []
        while q:
            top = q.popleft()
            if top[1] != cur_layer: # 当进入下一层时，结果res加一个新的list。
                res.append([top[0].val])
                cur_layer = top[1]
            else:
                res[-1].append(top[0].val)
            if top[0].left:
                q.append([top[0].left, cur_layer + 1])
            if top[0].right:
                q.append([top[0].right, cur_layer + 1])
        return res
    
    
# 思路二BFS：每次遍历一层
class Solution:
    def levelOrder(self, root: TreeNode) -> List[List[int]]:
        if not root:
            return []
        res = []
        next_layer = [root]
        while next_layer: 
            res.append([])
            cur_layer = next_layer
            next_layer = []
            for node in cur_layer: # 遍历当前层，存储下一层要遍历的结果。
                res[-1].append(node.val)
                if node.left:
                    next_layer.append(node.left)
                if node.right:
                    next_layer.append(node.right)
        return res

## 树的搜索

广搜、深搜：是对于**非线性结构**的遍历！！！

参考资料：
https://developer.aliyun.com/article/756316

迭代和递归实现
- DFS 遍历使用递归
- BFS 遍历使用队列数据结构。套路：
    - 先把根节点入队列。
    - 队列：存储需要待遍历的点
    - 每次从队列里取出一个元素，对结果进行更新操作。
    - 判断是否要把左右子节点入队列



深度优先遍历一句话总结

> 只要前面有可以走的路，就会一直向前走，直到无路可走才会回头

「无路可走」有两种情况：

- 遇到了墙；
- 遇到了已经走过的路；
  

BFS 的应用
- 层序遍历
- 最短路径

DFS的应用
- 是否有可行解，走迷宫


In [ ]:
# BFS Implementation
class 
from collections import deque
def BFS(root):
    if not root:
        return []
    res = []
    to_search = deque([root])
    while to_search:
        head = to_search.popleft()
        res.append(head.val)
        if head.left:
            to_search.append(head.left)
        if head.right:
            to_search.append(head.right)
    return res

# 伪代码，一般套路
# def BFS(root):
#     if isnull(root):
#         return
#     res = set_default_result(res)
#     push_node_in_queue(root, queue)
#     while is_not_empty(queue):
#         head = get_head(queue)
#         update_result(head, res)
#         if head.left:
#             push_node_in_queue(head.left, queue)
#         if head.right:
#             push_node_in_queue(head.right, queue)
#     return res

In [ ]:
# DFS Recursion Implementation
def dfs(root):
    if not root:
        return []
    res = []
    res.extend(self.dfs(root.left))
    res.extend(self.dfs(root.right))
    res.append(root.val)
    return res

# iteration implementation
def dfs(root):
    if not root:
        return
    stack = [root]
    while stack:
        head = stack.pop()
        process(head)
        if head.right: # 先把右节点入栈
            stack.append(head.right)
        if head.left: # 再把左节点入栈
            stack.append(head.left)
        

# 伪代码，递归一般套路
# def dfs(root):
#     if isnull(root):
#         return
#     process(root)
#     dfs(root.left)
#     dfs(root.right)

# Leetcode例题

## 树的遍历


- [(easy)897increasing order search tree](https://leetcode.cn/problems/increasing-order-search-tree/)。思路：前序遍历的变式。区别在于：使用只有右节点的树节点，而非数组，来保存遍历结果。
- [(medium)105Construct Binary Tree From Preorder And Inorder Traversal](https://leetcode.cn/problems/construct-binary-tree-from-preorder-and-inorder-traversal/)
    - 思路一：递归构造，每次在中序中找到当前根节点的位置，再左右子树递归构造。时间复杂度最坏情况是O(n^2)，当只有左子树的时候。耗时的地方是：寻找当前根节点的位置。
    - 思路二：记录前序节点到中序的位置映射关系，再递归构造。时间复杂度是O(n)
    - [ ] 💡思路三：迭代法：[参考题解](https://leetcode.cn/problems/construct-binary-tree-from-preorder-and-inorder-traversal/solution/cong-qian-xu-yu-zhong-xu-bian-li-xu-lie-gou-zao-9/)
- [(medium)106Construct Binary Tree From Inorder And Postorder Traversal](https://leetcode.cn/problems/construct-binary-tree-from-inorder-and-postorder-traversal/)
    - 思路一：递归构造，借助hashmap减少复杂度。
    - [ ] 💡思路二：迭代法，类似于105的迭代法。
- [(medium)889Construct Binary Tree From Preorder And Postorder Traversal](https://leetcode.cn/problems/construct-binary-tree-from-preorder-and-postorder-traversal/)。
    - 思路一：递归构造+hashmap。关键在于：通过两个数组，确定根，左子树对应的数组，和右子树对应的数组。如何确定呢？在前序遍历中，左子树的根是前序遍历的第二个数字，在后序遍历中寻找该数字的位置，即可知道左子树有几个节点，然后从前序和后序中，截取左右子树对应的子段，递归求解。
    - [ ] 💡思路二：迭代法，[参考题解的回复](https://leetcode.cn/problems/construct-binary-tree-from-preorder-and-postorder-traversal/solution/gen-ju-qian-xu-he-hou-xu-bian-li-gou-zao-er-cha-sh/)
    

In [ ]:
# 105 思路一：递归构造。时间复杂度最坏情况是O(n^2)
class Solution:
    def buildTree(self, preorder: List[int], inorder: List[int]) -> TreeNode:
        if not preorder:
            return None
        left_cnt = 0
        for node in inorder: # 这一步比较耗时 
            if node == preorder[0]:
                break
            left_cnt += 1
        root = TreeNode(preorder[0])
        root.left = self.buildTree(preorder[1:1+left_cnt], inorder[:left_cnt])
        root.right = self.buildTree(preorder[1+left_cnt:], inorder[1+left_cnt:])
        return root


# 思路二：先建立前序到中序的下标对应关系，时间复杂度是O(n)
class Solution:
    def recurseBuild(self, preorder, inorder, pre2in, diff):
        if not preorder:
            return None
        root = TreeNode(preorder[0])
        left_cnt = pre2in[preorder[0]] - diff
        root.left = self.recurseBuild(preorder[1:1+left_cnt], inorder[:left_cnt], pre2in, diff)
        root.right = self.recurseBuild(preorder[1+left_cnt:], inorder[1+left_cnt:], pre2in, 1 + left_cnt + diff)
        return root
        
    def buildTree(self, preorder: List[int], inorder: List[int]) -> TreeNode:
        pre2in = {}
        for i, node in enumerate(inorder):
            pre2in[node] = i
        return self.recurseBuild(preorder, inorder, pre2in, 0)
    
# 思路三：迭代：待学习

In [ ]:
# 106 思路一：递归调用
class Solution:
    def recurseBuild(self, inorder, postorder, post2in, diff):
        if not inorder:
            return None
        left_cnt = post2in[postorder[-1]] - diff
        root = TreeNode(postorder[-1])
        root.left = self.recurseBuild(inorder[:left_cnt], postorder[:left_cnt], post2in, diff)
        root.right = self.recurseBuild(inorder[left_cnt+1: ], postorder[left_cnt: -1], post2in, diff + left_cnt + 1)
        return root

    def buildTree(self, inorder: List[int], postorder: List[int]) -> TreeNode:
        post2in = {}
        for i, node in enumerate(inorder):
            post2in[node] = i
        return self.recurseBuild(inorder, postorder, post2in, 0)
# 思路二：迭代：待学习

In [ ]:
# 889 思路一：递归构造
class Solution:
    def recurseBuild(self, preorder, postorder, diff):
        if not preorder:
            return None
        root = TreeNode(preorder[0])
        if len(preorder) == 1: 
            return root
        left_cnt = self.pre2post[preorder[1]] - diff + 1 # 左子树的根是前序遍历的第二个数字，在后序遍历中寻找该数字的位置
        # 从前序和后序中，截取左右子树对应的子段，递归求解
        root.left = self.recurseBuild(preorder[1:1+left_cnt], postorder[:left_cnt], diff)
        root.right = self.recurseBuild(preorder[1+left_cnt:], postorder[left_cnt:-1], diff + left_cnt)
        return root

    def constructFromPrePost(self, preorder: List[int], postorder: List[int]) -> TreeNode:
        self.pre2post = {}
        for i, node in enumerate(postorder):
            self.pre2post[node] = i
        return self.recurseBuild(preorder, postorder, 0)
    
# 思路二：迭代解法，待学习
class Solution {
public:
    # 前序遍历和后序遍历访问顺序本质区别
    # 前序:走一步，记录一步.  后序: 每次走到头，回一步，记录一步。
    # 所以直接利用前序建树，利用后序返回，相当于前序递归访问，利用后序遍历返回上一节点
    # 将后序遍历充当前序遍历中的返回栈
    TreeNode* constructFromPrePost(vector<int>& pre, vector<int>& post) {
        vector<TreeNode*> stack;
        stack.push_back(new TreeNode(pre[0]));
        for(int i = 1, j = 0; i < pre.size(); ++i){
            TreeNode* node = new TreeNode(pre[i]);
            while(stack.back()->val == post[j])
                stack.pop_back(),j++;
            if(stack.back()->left == nullptr) stack.back()->left = node;
            else stack.back()->right = node;
            stack.push_back(node);
        }
        return stack[0];
    }
};

## 树的搜索

[参考题解](https://leetcode.cn/problems/binary-tree-level-order-traversal/solution/bfs-de-shi-yong-chang-jing-zong-jie-ceng-xu-bian-l/)
[wechat article](https://mp.weixin.qq.com/s?__biz=MzA5ODk3ODA4OQ==&mid=2648167208&idx=1&sn=d8118c7c0e0f57ea2bdd8aa4d6ac7ab7&chksm=88aa236ebfddaa78a6183cf6dcf88f82c5ff5efb7f5c55d6844d9104b307862869eb9032bd1f&token=1064083695&lang=zh_CN#rd)

### BFS
- [(easy)637Average of Levels in Binary Tree](https://leetcode.cn/problems/average-of-levels-in-binary-tree/)
    - 思路：BFS。队列中存放的是一层的所有节点，以层为单位进行操作，而非以单个节点为单位。
- [(medium)103Binary Tree Zigzag Level Order Traversal](https://leetcode.cn/problems/binary-tree-zigzag-level-order-traversal/)。思路：标准的bfs，区别在于每一层存储结果的时候，根据指示逆序即可。
- [(medium)199Binary Tree Right Side View](https://leetcode.cn/problems/binary-tree-right-side-view/)。思路：BFS。每层所有的节点都要存储，用于下一层遍历，因为有可能该层最右边的节点没有子节点，即该层任意一个节点都可能是下一层最右边节点的父节点。在保留该层结果的时候，保留最右边的值即可。
- [(medium)515Find Largest Value in Each Tree Row](https://leetcode.cn/problems/find-largest-value-in-each-tree-row/)。BFS：在保存每层的结果的时候，取最大值。
- [(medium)54201 Matrix](https://leetcode.cn/problems/01-matrix/)
    - 思路一：从每个点出发，进行bfs，求最小距离。超时。时间复杂度是O(m2n2)的
    - 思路二：先找到所有的根节点（矩阵值为0的节点），进行BFS。在找邻居的时候，找到一个邻居就更新距离，以减少重复寻找。
    
- [(medium)994Rotting Oranges](https://leetcode.cn/problems/rotting-oranges/)
    - 思路：BFS，需要额外检查是否所有的新鲜橙子都烂掉了。
- [(medium)1162As Far From Land As Possible](https://leetcode.cn/problems/as-far-from-land-as-possible/)
    - 思路：BFS，离水最近的陆地的最大距离。如果从水出发，每个水都能找到这么个距离，遍历所有水，求最大值即可。这样复杂度比较高。换种思路，如果从所有的陆地出发，就等价于最少用多大的距离，从陆地能到达所有的水。

- [(hard)126单词接龙II](https://leetcode.cn/problems/word-ladder-ii/)
    - 思路一：整体想法是建树，然后BFS：找到根节点(beginWord)到结束节点(endWord)的最短距离。一个节点的子节点是所有和它相差一个字母，且没有被用过的节点。递归搜索，直到达到endword。
    - 思路二：双向BFS，从beginWord向下找，从endWord向上找，如果有重合的就退出。

In [2]:
# 637 BFS
from collections import deque
class Solution:
    def averageOfLevels(self, root: Optional[TreeNode]) -> List[float]:
        q = deque([[root]])
        res = []
        while q:
            nodes = q.popleft()
            cur_layer_sum = 0
            next_layer_nodes = []
            for node in nodes:
                cur_layer_sum += node.val
                if node.left:
                    next_layer_nodes.append(node.left)
                if node.right:
                    next_layer_nodes.append(node.right)
            res.append(cur_layer_sum / 1.0 / len(nodes))
            if next_layer_nodes:
                q.append(next_layer_nodes)
        return res

NameError: name 'Optional' is not defined

In [ ]:
# 103 zigzag
from collections import deque
class Solution:
    def zigzagLevelOrder(self, root: TreeNode) -> List[List[int]]:
        left2right = 1
        if not root:
            return []
        q = deque([[root]])
        res = []
        while q:
            cur_layer = q.popleft()
            next_layer = []
            cur_layer_res = []
            for node in cur_layer:
                cur_layer_res.append(node.val)
                if node.left:
                    next_layer.append(node.left)
                if node.right:
                    next_layer.append(node.right)
            # 和标准bfs的区别：仅在于存储该层结果时，根据指示来存储
            if left2right == 1:
                res.append(cur_layer_res)
            else:
                res.append(cur_layer_res[::-1])
            left2right = 1 - left2right
            if next_layer:
                q.append(next_layer)
        return res

In [ ]:
# 199
class Solution:
    def rightSideView(self, root: Optional[TreeNode]) -> List[int]:
        if not root:
            return []
        res = []
        next_layer = [root]
        while next_layer:
            cur_layer = next_layer
            cur_layer_val = []
            next_layer = []
            for node in cur_layer:
                cur_layer_val.append(node.val)
                if node.left:
                    next_layer.append(node.left)
                if node.right:
                    next_layer.append(node.right) 
            # 和标准bfs的区别：在保留该层结果的时候，保留最右边的值即可。
            res.append(cur_layer_val[-1])
        return res

In [ ]:
# 515
class Solution:
    def largestValues(self, root: Optional[TreeNode]) -> List[int]:
        if not root:
            return []
        next_layer = [root]
        res = []
        while next_layer:
            cur_layer = next_layer
            next_layer = []
            cur_layer_res = []
            for node in cur_layer:
                cur_layer_res.append(node.val)
                if node.left:
                    next_layer.append(node.left)
                if node.right:
                    next_layer.append(node.right)
            # 和标准bfs的区别：在保存每层的结果的时候，取最大值。
            res.append(max(cur_layer_res))
        return res

In [ ]:
# 542 从每个点出发，进行bfs。超时
class Solution:
    def GetNeighsCheckZero(self, i, j, m, n, mat):
        neighs = []
        for next_i, next_j in [[i - 1,j], [i + 1, j], [i, j - 1], [i, j+1]]:
            if next_i < 0 or next_i >= m or next_j < 0 or next_j >= n:
                continue
            if mat[next_i][next_j] == 0:
                return [], True
            neighs.append([next_i, next_j])
        return neighs, False
            
    def updateMatrix(self, mat: List[List[int]]) -> List[List[int]]:
        m = len(mat)
        n = len(mat[0])
        res = []
        for i in range(m):
            layer_res = []
            for j in range(n):
                if mat[i][j] == 0:
                    layer_res.append(0)
                    continue
                node_dist = 0
                next_nodes = [[i, j]]
                while next_nodes:
                    node_dist += 1
                    cur_nodes = next_nodes
                    next_nodes = []
                    for node in cur_nodes:
                        neighbors, got_zero = self.GetNeighsCheckZero(node[0], node[1], m, n, mat)
                        if got_zero:
                            next_nodes = []
                            break
                        next_nodes.extend(neighbors)
                layer_res.append(node_dist)
            res.append(layer_res)
        return res
    
# 思路二：先找到所有的根节点，然后bfs
class Solution:
    def initialize(self, mat, m, n):
        # 找到所有的根节点：矩阵值为0的节点
        dist = []
        roots = []
        for i in range(m):
            dist.append([])
            for j in range(n):
                if mat[i][j] == 0:
                    roots.append([i,j])
                    dist[-1].append(0)
                else:
                    dist[-1].append(-1)
        return dist, roots
    
    def findNeighs(self, node, dist, cur_dist, m, n):
        i, j = node[0], node[1]
        neighs = []
        for next_i, next_j in [[i-1, j], [i + 1, j], [i, j - 1], [i, j + 1]]:
            if next_i < 0 or next_i >= m or next_j < 0 or next_j >= n:
                continue
            if dist[next_i][next_j] < 0:
                neighs.append([next_i, next_j])
                dist[next_i][next_j] = cur_dist # 在找到邻居的时候就更新dist矩阵，减少重复
        return neighs
            
            
    def updateMatrix(self, mat: List[List[int]]) -> List[List[int]]:
        m = len(mat)
        n = len(mat[0])
        dist, roots = self.initialize(mat, m, n)
        next_layer = roots
        cur_dist = 0
        while next_layer:
            cur_dist += 1
            cur_layer = next_layer
            next_layer = []
            for node in cur_layer:
                neighs = self.findNeighs(node, dist, cur_dist, m , n)
                next_layer.extend(neighs)
        return dist

In [ ]:
# 994
class Solution:
    def initalize(self, grid, m, n):
        cnt = 0
        roots = []
        for i in range(m):
            for j in range(n):
                if grid[i][j] == 1:
                    cnt += 1
                if grid[i][j] == 2:
                    roots.append([i, j])
        return cnt, roots

    def rotten(self, node, grid, m, n):
        neighs = []
        for next_i, next_j in [[node[0] -1 , node[1]], [node[0]+1, node[1]], [node[0], node[1] - 1], [node[0], node[1] + 1]]:
            if next_i < 0 or next_i >= m or next_j < 0 or next_j>= n or grid[next_i][next_j] != 1:
                continue
            grid[next_i][next_j] = 2
            neighs.append([next_i, next_j])
        return neighs

    def orangesRotting(self, grid: List[List[int]]) -> int:
        m = len(grid)
        n = len(grid[0])
        fresh_cnt, next_layer = self.initalize(grid, m, n)
        minutes = -1
        rotten_cnt = 0 # 和fresh_cnt一起，用于检查是否所有的新鲜橙子都烂掉了
        while next_layer:
            cur_layer = next_layer
            minutes += 1 # 更新时间
            next_layer = []
            for orange in cur_layer:
                new_rottens = self.rotten(orange, grid, m, n)
                next_layer.extend(new_rottens)
                rotten_cnt += len(new_rottens)
        if fresh_cnt == rotten_cnt:
            return max(minutes, 0)
        return -1

In [ ]:
# 1162
class Solution:
    def findAllLand(self, grid, n):
        lands = []
        for i in range(n):
            for j in range(n):
                if grid[i][j] == 1:
                    lands.append([i, j])
        return lands

    def findNeighs(self, land, grid, n):
        i, j = land[0], land[1]
        neighs = []
        for next_i, next_j in [[i-1,j], [i+1,j], [i, j-1], [i, j+1]]:
            if next_i < 0 or next_i >= n or next_j < 0 or next_j >= n or grid[next_i][next_j] == 1:
                continue
            neighs.append([next_i, next_j])
            grid[next_i][next_j] = 1 # 将水更新为陆地
        return neighs

    def maxDistance(self, grid: List[List[int]]) -> int:
        n = len(grid)
        next_layer = self.findAllLand(grid, n)
        if len(next_layer) == n*n:
            return -1
        dist = -1
        while next_layer:
            cur_layer = next_layer
            next_layer = []
            dist += 1 # 更新距离
            for land in cur_layer:
                water_neighs = self.findNeighs(land, grid, n)
                next_layer.extend(water_neighs)
        return dist

In [ ]:
# 126 思路一：单向BFS
from collections import deque
class Solution:
    def __init__(self):
        self.node2children = {}
        self.visited = {}
        self.end = None
    
    def OneDiff(self, word1, word2):
        diff_cnt = 0
        for i, c in enumerate(word1):
            if c != word2[i]:
                diff_cnt += 1
        return diff_cnt == 1
    
    def BuildChildren(self, beginWord, wordList, endWord):
        self.node2children[beginWord] = []
        self.visited[beginWord] = False
        for word in wordList:
            if self.OneDiff(beginWord, word):
                self.node2children[beginWord].append(word)
        for word in wordList:
            if word == endWord:
                continue
            self.node2children[word] = []
            self.visited[word] = False
            for other_word in wordList:
                if self.OneDiff(word, other_word):
                    self.node2children[word].append(other_word)

    def BFS(self, root, former_path):
        next_layer = [[root, former_path]]
        found_end = False
        while True: # 一层一层的广搜
            cur_layer = next_layer
            next_layer = []
            for node, former_path in cur_layer:
                # print(node, former_path)
                self.visited[node] = True
                for child in self.node2children[node]:
                    if child == self.end:
                        found_end = True
                    elif self.visited[child]: # 跳过已经遍历的节点
                        continue
                    next_layer.append([child, former_path + [child]])
            if found_end: # 当第一次遇到endWord的时候，就退出，这是最短路径
                return [path[1] for path in next_layer if path[1][-1] == self.end]
            if not next_layer:
                break
        return []

    def findLadders(self, beginWord: str, endWord: str, wordList: List[str]) -> List[List[str]]:
        if endWord not in wordList:
            return []
        self.end = endWord
        self.BuildChildren(beginWord, wordList, endWord) # 构建父节点和子节点之间的关系
        return self.BFS(beginWord, [beginWord]) # 广搜

In [ ]:
# 126 思路二：双向BFS
from collections import deque
class Solution:
    def __init__(self):
        self.node2children = {}
        self.begin_visited = {}
        self.end_visited = {}
    
    def OneDiff(self, word1, word2):
        diff_cnt = 0
        for i, c in enumerate(word1):
            if c != word2[i]:
                diff_cnt += 1
        return diff_cnt == 1
    
    def BuildChildren(self, wordList):
        for word in wordList:
            self.node2children[word] = []
            self.begin_visited[word] = False
            self.end_visited[word] = False
            for other_word in wordList:
                if self.OneDiff(word, other_word):
                    self.node2children[word].append(other_word)

    def mergePaths(self, meet_nodes, former_paths, end_paths):
        res = []
        for meet_node in meet_nodes:
            former_has = []
            later_has = []
            for p in former_paths:
                if p[-1] == meet_node:
                    former_has.append(p)
            for p in end_paths:
                if p[0] == meet_node:
                    later_has.append(p[1:])
            for fp in former_has:
                for lp in later_has:
                    res.append(fp+lp)
        return res


    def BFS(self, begin_path, end_path):
        begin_next_layer = [begin_path]
        end_next_layer = [end_path]
        meet_nodes = []
        while True:
            begin_cur_layer = begin_next_layer
            end_cur_layer = end_next_layer
            begin_next_layer, end_next_layer = [], []
            cur_visit = set() # 记录当前访问了哪些点
            for former_path in begin_cur_layer:
                node = former_path[-1]
                if self.end_visited[node]:
                    meet_nodes.append(node)
                else:
                    for child in self.node2children[node]:
                        if self.begin_visited[child]:
                            continue
                        cur_visit.add(child)
                        begin_next_layer.append(former_path + [child])
            for child in cur_visit:
                self.begin_visited[child] = True # 将访问的点状态进行更新
            if meet_nodes:
                return self.mergePaths(set(meet_nodes), begin_cur_layer, end_cur_layer)
            cur_visit = set()
            for former_path in end_cur_layer:
                node = former_path[0]
                if self.begin_visited[node]:
                    meet_nodes.append(node)
                else:
                    for child in self.node2children[node]:
                        if self.end_visited[child]:
                            continue
                        cur_visit.add(child)
                        end_next_layer.append([child] + former_path)
            for child in cur_visit:
                self.end_visited[child] = True
                        
            if meet_nodes:
                return self.mergePaths(set(meet_nodes), begin_next_layer, end_cur_layer)
            if not begin_next_layer and not end_next_layer:
                return []

    def findLadders(self, beginWord: str, endWord: str, wordList: List[str]) -> List[List[str]]:
        if endWord not in wordList:
            return []
        all_words = wordList + [beginWord]
        if beginWord in wordList:
            all_words = wordList
        self.BuildChildren(all_words)
        self.begin_visited[beginWord] = True
        self.end_visited[endWord] = True
        return self.BFS([beginWord], [endWord])

## DFS

- [(easy)543Diameter of Binary Tree](https://leetcode.cn/problems/diameter-of-binary-tree/)
    - 思路：先观察一个路径有什么特点？一个路径，总有一个根节点，根节点到最深左叶子节点的距离 + 根节点到最深右叶子节点的距离，就是路径的长度。这个根节点不必须为全树的真正根节点，可以是树中的任意节点。所以，我们对树中的任意节点需要两个信息：其左分支的最大深度，其右分支的最大深度，两者求和来更新全局的diameter最大值，两者求max来返回给父节点。很显然，这是个dfs，返回值是该节点的最大深度，维护一个全局变量，在递归中更新。
    
- [(medium)669Trim a Binary Search Tree](https://leetcode.cn/problems/trim-a-binary-search-tree/)。首先要想清楚，对当前节点，我要做哪些操作：我要对它的左子树和右子树进行剪枝，而不是对其本身进行剪枝，因为如果对其本身进行剪枝，需要替换它的父节点中该节点对应的分支。对其本身进行剪枝的操作，留给它父节点来做。为此，需要创建一个dummy 根节点，用于把根节点作为它的左子节点。如何对子节点进行剪枝？
    - 当子节点的值小于low的时候，就取子节点的右子节点；
    - 当子节点的值大于high的时候，就取子节点的左子节点；
  注意递归结束的条件：节点为空。
 
- 🌟[(medium)99Recover Binary Search Tree](https://leetcode.cn/problems/recover-binary-search-tree/)
    - 思路：DFS+中序遍历
    
- [(medium)1372Longest Zigzag Path in a Binary Tree](https://leetcode.cn/problems/longest-zigzag-path-in-a-binary-tree/)
    - 思路：DFS。
        - 对每个节点做什么操作？拿到两个子节点的值，更新自己的值。有两个值要更新：
            - 从该节点向左的zigzag路径最长值，使用左节点的右zigzag路径+1；
            - 从该节点向右的zigzag路径最长值，使用右节点的左zigzag路径+1。
        - 同时，在dfs的过程中更新全局变量
        - 递归终止的条件是什么？节点为空。这一步，可以加个early stop的判断，能减少些时间。
- [(medium)1367Linked List in Binary Tree](https://leetcode.cn/problems/linked-list-in-binary-tree/)
    - 思路一：对树中的每个点，做两种dfs展开：假设它是head的节点（从头开始）/或接着当前head（接着从head开始和父节点们相等的下一个node开始）。我这里对左子节点和右子节点分别尝试这两种情况。时间复杂度很高。
    - 思路二：整体思路还是尝试做两种dfs展开，但是顺序有所调整，并且加上了提前剪枝。区分开来这两种dfs：
        - 一种是：从原始head开始做的dfs。回头尝试（将该节点和原始的head对齐）的探索在这里做。
        - 另一种是：判断当前root开始是否是当前head开始的子树。如果当前root的值和head的值不一样，即可提前停止。这一种dfs里不做任何回头（将该节点和原始的head对齐）操作。
        而且在顺序上，是先第二种，再第一种。
- [(hard)297Serialize and Deserialize Binary Tree](https://leetcode-cn.com/problems/serialize-and-deserialize-binary-tree/)。主要思想用反序列化的方式来指导序列化。
    - 思路一：序列化的时候用前序遍历，反序列化用DFS+括号匹配。括号匹配是用来寻找左子树的字符串和右子树的字符串，从而实现递归构造。时间复杂度平均状态下是nlogn的，每一步确定左右子串，需要遍历平均一半长度的序列化后的字符串，平均要遍历logn次。
    - 思路二：反序列化直接遍历，用栈来存储当前需要构造子节点的节点。可以将时间复杂度降为n。具体细节为：
        - 栈中存储的对象是[节点，状态]，节点表示当前需要构造左子节点或右子节点的节点，0表示需要构造左子节点，1表示需要构造右子节点。
        - 对于遍历中的元素，需要做的操作是：
            - 如果是空，就弹出栈顶元素，根据栈顶元素中的状态，将其作为栈顶元素的左或右子节点。
                - 如果是作为左子节点，则重新将栈顶元素入栈，状态改为1
                - 如果是作为右子节点，则将父节点作为新的子节点，继续出栈顶，重复添加子节点的操作。
            - 如果非空，就新建一个节点，入栈，并将状态设置为0.
- [(hard)332Reconstruct Itinerary](https://leetcode.cn/problems/reconstruct-itinerary/)
    - 思路一：DFS+贪心。从当前机场，在所有可能的下一站中取字母序最前的机场，进行DFS。如果找到了一个，那必定是全局字母序最前的，即可返回。复杂的一点是：如何找到所有可能的下一站机场？我这里选择了用两个字典来维护：一个表示可以去哪些机场，一个表示已经去了哪些机场，差值就是剩下可以去的机场。
    - [ ] 思路二：欧拉路径，待学习，参考笔记树和图（三：图）


- [(hard)987Vertical Order Traversal of a Binary Tree](https://leetcode.cn/problems/vertical-order-traversal-of-a-binary-tree/)
- [(hard)剑指offer37 序列化二叉树](https://leetcode.cn/problems/xu-lie-hua-er-cha-shu-lcof/)
- [(medium)面试题04.06后继者](https://leetcode.cn/problems/successor-lcci/)

In [ ]:
# 543
class Solution:
    def __init__(self):
        self.max = -1
    def dfs(self, node):
        if not node:
            return -1
        left_depth = self.dfs(node.left)
        right_depth = self.dfs(node.right)
        self.max = max(self.max, left_depth + right_depth + 2)
        return max(left_depth, right_depth) + 1

    def diameterOfBinaryTree(self, root: Optional[TreeNode]) -> int:
        if not root:
            return -1
        self.dfs(root)
        return self.max

In [ ]:
# 669
class Solution:
    def DFSTrim(self, head, low, high):
        if not head: # 递归结束的条件：节点为空。
            return head
        head.left = self.DFSTrim(head.left, low, high)
        head.right = self.DFSTrim(head.right, low, high)
        if head.left: 
            if head.left.val < low: # 当子节点的值小于low的时候，就取子节点的右子节点；
                head.left = head.left.right
            elif head.left.val > high: # 当子节点的值大于high的时候，就取子节点的左子节点；
                head.left = head.left.left
        if head.right:
            if head.right.val > high:
                head.right = head.right.left
            elif head.right.val < low:
                head.right = head.right.right
        return head


    def trimBST(self, root: Optional[TreeNode], low: int, high: int) -> Optional[TreeNode]:
        dummy_head = TreeNode(left=root) # 需要创建一个dummy 根节点，用于把根节点作为它的左子节点
        self.DFSTrim(dummy_head, low, high)
        return dummy_head.left

In [ ]:
# 99 DFS + 中序遍历
class Solution:
    def __init__(self):
        self.left_node = None
        self.right_node = None
    
    def dfs(self, node, lower_bound, higher_bound):
        if not node:
            return
        # 先处理左节点
        self.dfs(node.left, lower_bound, node)
        # 中序遍历，对本节点的处理，放在左节点和右节点之间。
        if lower_bound:
            if node.val < lower_bound.val:
                if not self.left_node:
                    self.left_node = lower_bound
                self.right_node = node
        if higher_bound:
            if node.val > higher_bound.val:
                if not self.left_node:
                    self.left_node = node
                self.right_node = higher_bound
        # 最后处理右节点
        self.dfs(node.right, node, higher_bound)

    def recoverTree(self, root: Optional[TreeNode]) -> None:
        """
        Do not return anything, modify root in-place instead.
        """
        self.dfs(root, None, None)
        self.left_node.val, self.right_node.val = self.right_node.val, self.left_node.val

In [ ]:
# 1372 DFS

class Solution:
    def __init__(self):
        self.max = 0
    def dfs(self, node):
        if not node:
            return -1, -1
        _, right_path = self.dfs(node.left) # 先子节点，再根节点。
        left_path, _  = self.dfs(node.right)
        self.max = max(self.max, right_path + 1, left_path + 1) # 更新全局变量
        return right_path + 1, left_path + 1

    def longestZigZag(self, root: TreeNode) -> int:
        self.dfs(root)
        return self.max
    
    
class Solution:
    def __init__(self):
        self.max = 0
    def dfs(self, node):
        right_path = -1
        if node.left: # 加个判断，early stop，降低些时间。
            _, right_path = self.dfs(node.left)
        left_path = -1
        if node.right:
            left_path, _  = self.dfs(node.right)
        self.max = max(self.max, right_path + 1, left_path + 1)
        return right_path + 1, left_path + 1

    def longestZigZag(self, root: TreeNode) -> int:
        self.dfs(root)
        return self.max

In [ ]:
# 1367 思路一：对每个节点做两种dfs展开

class Solution:
    def isSubPath(self, head: ListNode, root: TreeNode) -> bool:
        print(head, root)
        if not head or (root.val == head.val and not head.next):
            return True
        if root.left:
            if root.val == head.val:
                if self.isSubPath(head.next, root.left):
                    return True       
            if self.isSubPath(head, root.left):
                return True        
        if root.right:
            if root.val == head.val:
                if self.isSubPath(head.next, root.right):
                    return True
            if self.isSubPath(head, root.right):
                return True
        return False
    
# 思路二：
class Solution:
    def dfs(self, cur_head, root):
        # 判断当前root开始是否是当前head开始的子树
        print("dfs: ", cur_head, root)
        if not cur_head:
            return True
        if not root:
            return False
        if root.val == cur_head.val:
            return self.dfs(cur_head.next, root.left) or self.dfs(cur_head.next, root.right)
        # 当前root的值和当前head的值不一样，即可提前剪枝。
        return False
    
    def isSubPath(self, head: ListNode, root: TreeNode) -> bool:
        # 从原始head开始做的dfs。
        print("isSubPath: ", head, root)
        if not head:
            return True
        if not root:
            return False
        return self.dfs(head, root) or self.isSubPath(head, root.left) or self.isSubPath(head, root.right)
        


In [ ]:
# 1367两种思路复杂度比较
# example
[1,4,2,6]
[1,4,4,null,2,2,null,1,null,6,8,null,null,null,null,1,3]


# one: 做了13次尝试
ListNode{val: 1, next: ListNode{val: 4, next: ListNode{val: 2, next: ListNode{val: 6, next: None}}}} TreeNode{val: 1, left: TreeNode{val: 4, left: None, right: TreeNode{val: 2, left: TreeNode{val: 1, left: None, right: None}, right: None}}, right: TreeNode{val: 4, left: TreeNode{val: 2, left: TreeNode{val: 6, left: None, right: None}, right: TreeNode{val: 8, left: TreeNode{val: 1, left: None, right: None}, right: TreeNode{val: 3, left: None, right: None}}}, right: None}}
ListNode{val: 4, next: ListNode{val: 2, next: ListNode{val: 6, next: None}}} TreeNode{val: 4, left: None, right: TreeNode{val: 2, left: TreeNode{val: 1, left: None, right: None}, right: None}}
ListNode{val: 2, next: ListNode{val: 6, next: None}} TreeNode{val: 2, left: TreeNode{val: 1, left: None, right: None}, right: None}
ListNode{val: 6, next: None} TreeNode{val: 1, left: None, right: None}
ListNode{val: 2, next: ListNode{val: 6, next: None}} TreeNode{val: 1, left: None, right: None}
ListNode{val: 4, next: ListNode{val: 2, next: ListNode{val: 6, next: None}}} TreeNode{val: 2, left: TreeNode{val: 1, left: None, right: None}, right: None}
ListNode{val: 4, next: ListNode{val: 2, next: ListNode{val: 6, next: None}}} TreeNode{val: 1, left: None, right: None}
ListNode{val: 1, next: ListNode{val: 4, next: ListNode{val: 2, next: ListNode{val: 6, next: None}}}} TreeNode{val: 4, left: None, right: TreeNode{val: 2, left: TreeNode{val: 1, left: None, right: None}, right: None}}
ListNode{val: 1, next: ListNode{val: 4, next: ListNode{val: 2, next: ListNode{val: 6, next: None}}}} TreeNode{val: 2, left: TreeNode{val: 1, left: None, right: None}, right: None}
ListNode{val: 1, next: ListNode{val: 4, next: ListNode{val: 2, next: ListNode{val: 6, next: None}}}} TreeNode{val: 1, left: None, right: None}
ListNode{val: 4, next: ListNode{val: 2, next: ListNode{val: 6, next: None}}} TreeNode{val: 4, left: TreeNode{val: 2, left: TreeNode{val: 6, left: None, right: None}, right: TreeNode{val: 8, left: TreeNode{val: 1, left: None, right: None}, right: TreeNode{val: 3, left: None, right: None}}}, right: None}
ListNode{val: 2, next: ListNode{val: 6, next: None}} TreeNode{val: 2, left: TreeNode{val: 6, left: None, right: None}, right: TreeNode{val: 8, left: TreeNode{val: 1, left: None, right: None}, right: TreeNode{val: 3, left: None, right: None}}}
ListNode{val: 6, next: None} TreeNode{val: 6, left: None, right: None}





# two: 做了10次尝试
isSubPath:  ListNode{val: 1, next: ListNode{val: 4, next: ListNode{val: 2, next: ListNode{val: 6, next: None}}}} TreeNode{val: 1, left: TreeNode{val: 4, left: None, right: TreeNode{val: 2, left: TreeNode{val: 1, left: None, right: None}, right: None}}, right: TreeNode{val: 4, left: TreeNode{val: 2, left: TreeNode{val: 6, left: None, right: None}, right: TreeNode{val: 8, left: TreeNode{val: 1, left: None, right: None}, right: TreeNode{val: 3, left: None, right: None}}}, right: None}}
dfs:  ListNode{val: 1, next: ListNode{val: 4, next: ListNode{val: 2, next: ListNode{val: 6, next: None}}}} TreeNode{val: 1, left: TreeNode{val: 4, left: None, right: TreeNode{val: 2, left: TreeNode{val: 1, left: None, right: None}, right: None}}, right: TreeNode{val: 4, left: TreeNode{val: 2, left: TreeNode{val: 6, left: None, right: None}, right: TreeNode{val: 8, left: TreeNode{val: 1, left: None, right: None}, right: TreeNode{val: 3, left: None, right: None}}}, right: None}}
dfs:  ListNode{val: 4, next: ListNode{val: 2, next: ListNode{val: 6, next: None}}} TreeNode{val: 4, left: None, right: TreeNode{val: 2, left: TreeNode{val: 1, left: None, right: None}, right: None}}
dfs:  ListNode{val: 2, next: ListNode{val: 6, next: None}} None
dfs:  ListNode{val: 2, next: ListNode{val: 6, next: None}} TreeNode{val: 2, left: TreeNode{val: 1, left: None, right: None}, right: None}
dfs:  ListNode{val: 6, next: None} TreeNode{val: 1, left: None, right: None}
dfs:  ListNode{val: 6, next: None} None
dfs:  ListNode{val: 4, next: ListNode{val: 2, next: ListNode{val: 6, next: None}}} TreeNode{val: 4, left: TreeNode{val: 2, left: TreeNode{val: 6, left: None, right: None}, right: TreeNode{val: 8, left: TreeNode{val: 1, left: None, right: None}, right: TreeNode{val: 3, left: None, right: None}}}, right: None}
dfs:  ListNode{val: 2, next: ListNode{val: 6, next: None}} TreeNode{val: 2, left: TreeNode{val: 6, left: None, right: None}, right: TreeNode{val: 8, left: TreeNode{val: 1, left: None, right: None}, right: TreeNode{val: 3, left: None, right: None}}}
dfs:  ListNode{val: 6, next: None} TreeNode{val: 6, left: None, right: None}
dfs:  None None
  

# example2
[1,1,1,1,1]
[1,1,1,null,1,1,null,1,null,1,1,null,null,null,null,1,1]

# one 做了21次尝试
ListNode{val: 1, next: ListNode{val: 1, next: ListNode{val: 1, next: ListNode{val: 1, next: ListNode{val: 1, next: None}}}}} TreeNode{val: 1, left: TreeNode{val: 1, left: None, right: TreeNode{val: 1, left: TreeNode{val: 1, left: None, right: None}, right: None}}, right: TreeNode{val: 1, left: TreeNode{val: 1, left: TreeNode{val: 1, left: None, right: None}, right: TreeNode{val: 1, left: TreeNode{val: 1, left: None, right: None}, right: TreeNode{val: 1, left: None, right: None}}}, right: None}}
ListNode{val: 1, next: ListNode{val: 1, next: ListNode{val: 1, next: ListNode{val: 1, next: None}}}} TreeNode{val: 1, left: None, right: TreeNode{val: 1, left: TreeNode{val: 1, left: None, right: None}, right: None}}
ListNode{val: 1, next: ListNode{val: 1, next: ListNode{val: 1, next: None}}} TreeNode{val: 1, left: TreeNode{val: 1, left: None, right: None}, right: None}
ListNode{val: 1, next: ListNode{val: 1, next: None}} TreeNode{val: 1, left: None, right: None}
ListNode{val: 1, next: ListNode{val: 1, next: ListNode{val: 1, next: None}}} TreeNode{val: 1, left: None, right: None}
ListNode{val: 1, next: ListNode{val: 1, next: ListNode{val: 1, next: ListNode{val: 1, next: None}}}} TreeNode{val: 1, left: TreeNode{val: 1, left: None, right: None}, right: None}
ListNode{val: 1, next: ListNode{val: 1, next: ListNode{val: 1, next: None}}} TreeNode{val: 1, left: None, right: None}
ListNode{val: 1, next: ListNode{val: 1, next: ListNode{val: 1, next: ListNode{val: 1, next: None}}}} TreeNode{val: 1, left: None, right: None}
ListNode{val: 1, next: ListNode{val: 1, next: ListNode{val: 1, next: ListNode{val: 1, next: ListNode{val: 1, next: None}}}}} TreeNode{val: 1, left: None, right: TreeNode{val: 1, left: TreeNode{val: 1, left: None, right: None}, right: None}}
ListNode{val: 1, next: ListNode{val: 1, next: ListNode{val: 1, next: ListNode{val: 1, next: None}}}} TreeNode{val: 1, left: TreeNode{val: 1, left: None, right: None}, right: None}
ListNode{val: 1, next: ListNode{val: 1, next: ListNode{val: 1, next: None}}} TreeNode{val: 1, left: None, right: None}
ListNode{val: 1, next: ListNode{val: 1, next: ListNode{val: 1, next: ListNode{val: 1, next: None}}}} TreeNode{val: 1, left: None, right: None}
ListNode{val: 1, next: ListNode{val: 1, next: ListNode{val: 1, next: ListNode{val: 1, next: ListNode{val: 1, next: None}}}}} TreeNode{val: 1, left: TreeNode{val: 1, left: None, right: None}, right: None}
ListNode{val: 1, next: ListNode{val: 1, next: ListNode{val: 1, next: ListNode{val: 1, next: None}}}} TreeNode{val: 1, left: None, right: None}
ListNode{val: 1, next: ListNode{val: 1, next: ListNode{val: 1, next: ListNode{val: 1, next: ListNode{val: 1, next: None}}}}} TreeNode{val: 1, left: None, right: None}
ListNode{val: 1, next: ListNode{val: 1, next: ListNode{val: 1, next: ListNode{val: 1, next: None}}}} TreeNode{val: 1, left: TreeNode{val: 1, left: TreeNode{val: 1, left: None, right: None}, right: TreeNode{val: 1, left: TreeNode{val: 1, left: None, right: None}, right: TreeNode{val: 1, left: None, right: None}}}, right: None}
ListNode{val: 1, next: ListNode{val: 1, next: ListNode{val: 1, next: None}}} TreeNode{val: 1, left: TreeNode{val: 1, left: None, right: None}, right: TreeNode{val: 1, left: TreeNode{val: 1, left: None, right: None}, right: TreeNode{val: 1, left: None, right: None}}}
ListNode{val: 1, next: ListNode{val: 1, next: None}} TreeNode{val: 1, left: None, right: None}
ListNode{val: 1, next: ListNode{val: 1, next: ListNode{val: 1, next: None}}} TreeNode{val: 1, left: None, right: None}
ListNode{val: 1, next: ListNode{val: 1, next: None}} TreeNode{val: 1, left: TreeNode{val: 1, left: None, right: None}, right: TreeNode{val: 1, left: None, right: None}}
ListNode{val: 1, next: None} TreeNode{val: 1, left: None, right: None}


    
# two dfs做了17次尝试
isSubPath:  ListNode{val: 1, next: ListNode{val: 1, next: ListNode{val: 1, next: ListNode{val: 1, next: ListNode{val: 1, next: None}}}}} TreeNode{val: 1, left: TreeNode{val: 1, left: None, right: TreeNode{val: 1, left: TreeNode{val: 1, left: None, right: None}, right: None}}, right: TreeNode{val: 1, left: TreeNode{val: 1, left: TreeNode{val: 1, left: None, right: None}, right: TreeNode{val: 1, left: TreeNode{val: 1, left: None, right: None}, right: TreeNode{val: 1, left: None, right: None}}}, right: None}}
dfs:  ListNode{val: 1, next: ListNode{val: 1, next: ListNode{val: 1, next: ListNode{val: 1, next: ListNode{val: 1, next: None}}}}} TreeNode{val: 1, left: TreeNode{val: 1, left: None, right: TreeNode{val: 1, left: TreeNode{val: 1, left: None, right: None}, right: None}}, right: TreeNode{val: 1, left: TreeNode{val: 1, left: TreeNode{val: 1, left: None, right: None}, right: TreeNode{val: 1, left: TreeNode{val: 1, left: None, right: None}, right: TreeNode{val: 1, left: None, right: None}}}, right: None}}
dfs:  ListNode{val: 1, next: ListNode{val: 1, next: ListNode{val: 1, next: ListNode{val: 1, next: None}}}} TreeNode{val: 1, left: None, right: TreeNode{val: 1, left: TreeNode{val: 1, left: None, right: None}, right: None}}
dfs:  ListNode{val: 1, next: ListNode{val: 1, next: ListNode{val: 1, next: None}}} None
dfs:  ListNode{val: 1, next: ListNode{val: 1, next: ListNode{val: 1, next: None}}} TreeNode{val: 1, left: TreeNode{val: 1, left: None, right: None}, right: None}
dfs:  ListNode{val: 1, next: ListNode{val: 1, next: None}} TreeNode{val: 1, left: None, right: None}
dfs:  ListNode{val: 1, next: None} None
dfs:  ListNode{val: 1, next: None} None
dfs:  ListNode{val: 1, next: ListNode{val: 1, next: None}} None
dfs:  ListNode{val: 1, next: ListNode{val: 1, next: ListNode{val: 1, next: ListNode{val: 1, next: None}}}} TreeNode{val: 1, left: TreeNode{val: 1, left: TreeNode{val: 1, left: None, right: None}, right: TreeNode{val: 1, left: TreeNode{val: 1, left: None, right: None}, right: TreeNode{val: 1, left: None, right: None}}}, right: None}
dfs:  ListNode{val: 1, next: ListNode{val: 1, next: ListNode{val: 1, next: None}}} TreeNode{val: 1, left: TreeNode{val: 1, left: None, right: None}, right: TreeNode{val: 1, left: TreeNode{val: 1, left: None, right: None}, right: TreeNode{val: 1, left: None, right: None}}}
dfs:  ListNode{val: 1, next: ListNode{val: 1, next: None}} TreeNode{val: 1, left: None, right: None}
dfs:  ListNode{val: 1, next: None} None
dfs:  ListNode{val: 1, next: None} None
dfs:  ListNode{val: 1, next: ListNode{val: 1, next: None}} TreeNode{val: 1, left: TreeNode{val: 1, left: None, right: None}, right: TreeNode{val: 1, left: None, right: None}}
dfs:  ListNode{val: 1, next: None} TreeNode{val: 1, left: None, right: None}
dfs:  None None



In [ ]:
# 297 思路一：前序遍历，DFS+括号匹配
# 执行用时：220 ms, 在所有 Python3 提交中击败了52% 的用户
# 内存消耗：24.5 MB, 在所有 Python3 提交中击败了。。。% 的用户
class Codec:

    def serialize(self, root):
        """Encodes a tree to a single string.
        
        :type root: TreeNode
        :rtype: str
        """
        if not root:
            return ''
        res = str(root.val)
        res += ('L(' + self.serialize(root.left) + ')') # 为了反序列化做括号匹配
        res += ('R(' + self.serialize(root.right) + ')')
        # print(root, res)
        return res
        

    def deserialize(self, data):
        """Decodes your encoded data to tree.
        
        :type data: str
        :rtype: TreeNode
        """
        if not data:
            return None
        # print(data)
        cur_val_id = 1
        while data[cur_val_id] != 'L':
            cur_val_id += 1
        cur_val = data[:cur_val_id]
        root = TreeNode(int(cur_val))
        cur_cnt = 1
        left_end_id = cur_val_id + 2
        while cur_cnt > 0:
            if data[left_end_id] == '(':
                cur_cnt += 1
            elif data[left_end_id] == ')':
                cur_cnt -= 1
            left_end_id += 1
        root.left = self.deserialize(data[cur_val_id + 2: left_end_id-1])
        root.right = self.deserialize(data[left_end_id + 2: -1])
        return root

# 括号匹配的变形：左右标志
# 执行用时：160 ms, 在所有 Python3 提交中击败了55% 的用户
# 内存消耗：22.3 MB, 在所有 Python3 提交中击败了6% 的用户
class Codec:
    def serialize(self, root):
        """Encodes a tree to a single string.
        
        :type root: TreeNode
        :rtype: str
        """
        if not root:
            return ''
        res = str(root.val)
        res += ('L' + self.serialize(root.left)) # 发现可以使用L或者R来替代括号，减少内存
        res += ('R' + self.serialize(root.right))
        return res
        

    def deserialize(self, data):
        """Decodes your encoded data to tree.
        
        :type data: str
        :rtype: TreeNode
        """
        if not data:
            return None
        # print(data)
        cur_val_id = 1
        while data[cur_val_id] != 'L':
            cur_val_id += 1
        cur_val = data[:cur_val_id]
        root = TreeNode(int(cur_val))
        
        cur_cnt = 1 # 括号匹配的思想，匹配L和R标志的个数。
        left_end_id = cur_val_id
        while cur_cnt > 0:
            left_end_id += 1
            if data[left_end_id] == 'L':
                cur_cnt += 1
            elif data[left_end_id] == 'R':
                cur_cnt -= 1
        root.left = self.deserialize(data[cur_val_id + 1: left_end_id])
        root.right = self.deserialize(data[left_end_id + 1: ])
        return root
    
# 思路二：用栈来反序列化
# 执行用时：108 ms, 在所有 Python3 提交中击败了92.18% 的用户
# 内存消耗：20.4 MB, 在所有 Python3 提交中击败了88.63% 的用户
class Codec:

    def serialize(self, root):
        """Encodes a tree to a single string.
        
        :type root: TreeNode
        :rtype: str
        """
        if not root:
            return ','
        res = (str(root.val) + ',')
        res += self.serialize(root.left)
        res += self.serialize(root.right)
        return res
        

    def deserialize(self, data):
        """Decodes your encoded data to tree.
        
        :type data: str
        :rtype: TreeNode
        """
        vals = data.split(',')[:-1]
        if len(vals) < 3:
            return []
        root = TreeNode(int(vals[0]))
        stack = [[root, 0]]
        for val in vals[1: ]:
            if val == '':
                cur_node = None
                while stack:
                    former_node, status = stack.pop()
                    if status == 1:
                        former_node.right = cur_node
                        cur_node = former_node
                    else:
                        former_node.left = cur_node
                        stack.append([former_node, 1])
                        break
            else:
                cur_node = TreeNode(int(val))
                stack.append([cur_node, 0])
        return root

In [ ]:
# 332 思路一：DFS建树
class Solution:
    def __init__(self):
        self.itineary = None
        self.num_tickets =  0
        self.Dep2ArrCnt = {} # 可以去哪些机场
        self.usedCnt = {} # 已经去了哪些机场
    
    def count(self, tickets):
        self.num_tickets = len(tickets)
        for depart, arriv in tickets:
            if depart not in self.Dep2ArrCnt:
                self.Dep2ArrCnt[depart] = {arriv : 1}
                self.usedCnt[depart] = {}
            else:
                self.Dep2ArrCnt[depart][arriv] = self.Dep2ArrCnt[depart].get(arriv, 0) + 1
            self.usedCnt[depart][arriv] = 0

    def DFSBuild(self, cur_itinerary):
        if len(cur_itinerary) == self.num_tickets + 1: # 走完了所有行程，返回
            self.itineary = cur_itinerary
            return
        cur_departure = cur_itinerary[-1]
        if cur_departure not in self.Dep2ArrCnt:
            return
        all_arrives = self.Dep2ArrCnt[cur_departure]
        possible = [] # 所有可能的下一站机场
        for arrive, cnt in all_arrives.items():
            if cur_departure in self.usedCnt and arrive in self.usedCnt[cur_departure] and self.usedCnt[cur_departure][arrive] < cnt:
                possible.append(arrive)
        for cur_arrive in sorted(possible): # 排序：贪心地选择当前节点所连的节点中字典序最小的那一个
            if self.itineary is not None: # early stop
                return
            self.usedCnt[cur_departure][cur_arrive] += 1
            self.DFSBuild(cur_itinerary + [cur_arrive])
            self.usedCnt[cur_departure][cur_arrive] -= 1
        
    def findItinerary(self, tickets: List[List[str]]) -> List[str]:
        self.count(tickets)
        self.DFSBuild(['JFK'])
        return self.itineary

## 迭代方法专项



# 我的总结

BFS解题重点：
1. 变式：以个为单位变为以层为单位
2. 变式：根节点从一个变为一list
3. 状态变量啥时候更新要注意，适当更新可减少重复访问。
    - 有的时候，在找子节点前更新状态变量，比如例题994中的minutes，1162中的dist，这种变量常常是结果要求的，初始化的值要和更新的位置相匹配。
    - 有的时候，在找子节点的时候更新状态变量，比如1162中的grid[next_i][next_j]。这种变量常常是用于记录访问状态的
    - 有的时候，在找完所有子节点之后再更新，比如126的双向BFS解法中的self.begin_visited和self.end_visited，这是因为要记录所有的路径，允许重复。
    
    
DFS解题重点：
1. 递归结束的条件是什么
2. 每一步要做什么操作，要返回什么值？
3. 常常需要一个全局变量（往往是最终的结果），在递归的过程中更新这个结果变量。
4. 和左右子节点的顺序如何？先对本节点做操作，再遍历左右子节点？还是先遍历左右子节点再对本节点做操作？先左还是先右？（遍历顺序，前序？中序？后序）参考例题99，先左，再本节点，最后右节点


在遍历中使用栈：参考例题297
- 弄清对遍历中的每个元素做什么操作
- 弄清栈中的元素等待做什么？什么情况入栈，什么情况出栈？